In [1]:
from pathlib import Path

def list_daily_station_files(root: Path = Path(r"c:\Users\Administrator\OneDrive\Documents\Github\2025_06_30"),
                             folder_name: str = "每日各站進出站人數",
                             exclude_names=("{manifest.csv}", "{schema.csv}")):
    root = Path(root)
    # 找到所有同名資料夾
    found_dirs = list(root.rglob(folder_name))
    if not found_dirs:
        print(f"找不到資料夾：{folder_name}，搜尋根目錄：{root}")
        return []

    # 正規化排除清單（小寫）
    exclude = {n.lower().strip("{}") for n in exclude_names}
    all_paths = []
    for d in found_dirs:
        files = [p.resolve() for p in d.rglob("*")
                 if p.is_file() and p.name.lower() not in exclude]
        files.sort()
        print(f"資料夾：{d}，找到 {len(files)} 個檔案（已排除 manifest/schema）")
        for f in files:
            print(f)
        all_paths.extend(files)

    # 移除重複並回傳
    unique_paths = sorted(dict.fromkeys(all_paths))
    return unique_paths

# 範例執行
paths = list_daily_station_files()
# 如果需要，paths 現在是所有絕對路徑的 list

資料夾：c:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數，找到 5 個檔案（已排除 manifest/schema）
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數20190423-20191231.csv
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數2020.csv
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數2021.csv
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數2022.csv
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數\每日各站進出站人數2023.csv


In [2]:
from pathlib import Path
from pprint import pprint

def list_daily_flow_files(base_dir: Path | None = None):
    """
    1. 取得 '每日各站進出站人數' 資料夾
    2. 取得其中所有 CSV 檔案的絕對路徑
    3. 排除 manifest.csv 與 schema.csv
    """
    if base_dir is None:
        base_dir = Path.cwd()  # 與 lesson19_2.ipynb 做法一致
    target_dir = base_dir / "每日各站進出站人數"
    if not target_dir.is_dir():
        raise FileNotFoundError(f"找不到資料夾: {target_dir}")

    excluded = {"manifest.csv", "schema.csv"}
    csv_paths = sorted(
        p.resolve()
        for p in target_dir.glob("*.csv")
        if p.name not in excluded
    )
    return target_dir.resolve(), csv_paths

folder_abs_path, file_abs_paths = list_daily_flow_files()

print("資料夾絕對路徑:")
print(folder_abs_path)
print("\n檔案絕對路徑 (已排除 manifest.csv, schema.csv):")
pprint([str(p) for p in file_abs_paths])

資料夾絕對路徑:
C:\Users\Administrator\OneDrive\Documents\Github\2025_06_30\lesson19\每日各站進出站人數

檔案絕對路徑 (已排除 manifest.csv, schema.csv):
['C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數20190423-20191231.csv',
 'C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數2020.csv',
 'C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數2021.csv',
 'C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數2022.csv',
 'C:\\Users\\Administrator\\OneDrive\\Documents\\Github\\2025_06_30\\lesson19\\每日各站進出站人數\\每日各站進出站人數2023.csv']
